## Luis Omar Leyva Navarrete - A01570367

# Instrucciones
### Dado:

* Habitación de MxN espacios.
* Número de agentes.
* Porcentaje de celdas inicialmente sucias.
* Tiempo máximo de ejecución.

### Realiza la siguiente simulación:
* Inicializa las celdas sucias (ubicaciones aleatorias).
* Todos los agentes empiezan en la celda [1,1].
* En cada paso de tiempo:
  * Si la celda está sucia, entonces aspira.
  * Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
* Se ejecuta el tiempo máximo establecido.

### Deberás recopilar la siguiente información durante la ejecución:
* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
* Porcentaje de celdas limpias después del termino de la simulación.
* Número de movimientos realizados por todos los agentes.
* Analiza cómo la cantidad de agentes impacta el tiempo dedicado, así como la cantidad de movimientos realizados. Desarrollar un informe con lo observado.

## Imports
Se utilizaran los siguientes paquetes de python
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para instalar mesa hay que hay que utilizar el siguiente comando:

In [1]:
!pip install mesa

Ahora importamos las librerias y funciones a utilizar.

In [2]:
# Importamos las clases para manejar el modelo y agente.
from mesa import Agent, Model 

# Utilizamos Multigrid para poder manejar varios agentes.
from mesa.space import MultiGrid

# Activamos a todos los agentes a la vez.
from mesa.time import SimultaneousActivation

# Obtenemos la información completa del grid para poder graficarlo.
from mesa.datacollection import DataCollector

# Esta libreria se utilizara en conjunto con data collector para graficar el grid.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Librerias para manejar operaciones numericas.
import numpy as np
import pandas as pd
import random
# librerias para obtener el tiempo de ejecucion.
import time
import datetime

In [3]:
def obtener_habitacion(model):
    habitacion = np.zeros((model.grid.width, model.grid.height))
    for celda in model.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
    return habitacion

class Movement(Agent):
    '''
    Clase para los movimientos de la aspiradora
    '''
    
    grid = None
    x = None
    y = None
    moore = False

    def __init__(self, pos, model, moore=False):
        super().__init__(pos, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        '''
        Mover la aspiradora en cualquier direccion
        '''
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, True)
        next_move = random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

class Suciedad(Agent):
    def __init__(self, pos, model):
        self.pos = pos

class Aspiradora(Movement):
    '''
    Agente que limpia el suelo
    '''

    def __init__(self, pos, model):
        super().__init__(pos, model)

    def step(self):
        self.random_move()

        # limpiar suciedad
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        sucio = [obj for obj in this_cell if isinstance(obj, Sheep)]
        if len(sucio) > 0:
            elim_suciedad = self.random.choice(sucio)

            self.model.grid._remove_agent(self.pos, elim_suciedad)
            self.model.schedule.remove(elim_suciedad)

class LimpiarSuciedad(Model):
    def __init__(self, width, height, suciedad_inicial, aspiradora_inicial):
        self.suciedad_inicial = suciedad_inicial
        self.aspiradora_inicial = aspiradora_inicial
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)

        #Crear suciedad
        for i in range(self.suciedad_inicial):
            x = self.random.randrange(width)
            y = self.random.randrange(height)
            suciedad = Suciedad((x,y),self)
            self.grid.place_agent(suciedad,(x,y))
            self.schedule.add(suciedad)

        #crear aspiradora
        for i in range(self.aspiradora_inicial):
            x = random.randrange(width)
            y = random.randrange(height)
            aspiradora = Aspiradora((x, y), self)
            self.grid.place_agent(aspiradora, (x, y))
            self.schedule.add(suciedad)

        self.datacollector = DataCollector(model_reporters={"Grid":get_grid})
    
    def step(self):
        '''
        Guardamos la informacion para luego graficarla
        '''
        self.datacollector.collect(self)
        self.schedule.step()


## Tiempo de ejecucion

In [4]:
Max_Time = 1
tiempo_init = str(datetime.timedelta(seconds=Max_Time))
model = LimpiarSuciedad(10,10,6,1)
while((time.time()-start_time) < Max_Time):
    model.step()

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Exception: Agent with unique id 5 already added to scheduler

## Graficar

## Crear Modelo